In [ ]:
from faircorels import *
import pandas as pd
import numpy as np
import time

In [ ]:
#Helper functions for computing fairness/accuracy
def compute_TPR_GAP(preds, Y, group):
    res = {}
    res['TPR'] = sum(preds[Y])/len(preds[Y])
    res['TPR_1'] = sum(preds[Y & group])/len(preds[Y & group])
    res['TPR_2'] = sum(preds[Y & ~group])/len(preds[Y & ~group])
    res['TPR_GAP'] = abs(res['TPR_1'] - res['TPR_2'])
    return res

def compute_TNR_GAP(preds, Y, group):
    res = {}
    res['TNR'] = sum(~preds[~Y])/len(preds[~Y])
    res['TNR_1'] = sum(~preds[~Y & group])/len(preds[~Y & group])
    res['TNR_2'] = sum(~preds[~Y & ~group])/len(preds[~Y & ~group])
    res['TNR_GAP'] = abs(res['TNR_1'] - res['TNR_2'])
    return res

def compute_ACC_GAP(preds, Y, group):
    res = {}
    res['ACC'] = sum(preds == Y)/len(Y)
    res['ACC_1'] = sum(preds[group] == Y[group])/len(Y[group])
    res['ACC_2'] = sum(preds[~group] == Y[~group])/len(Y[~group])
    res['ACC_GAP'] = abs(res['ACC_1'] - res['ACC_2'])
    return res
    
def compute_EqOpp(preds, Y, group):
    return compute_TPR_GAP(preds, Y, group)

def compute_EqOd(preds, Y, group):
    return compute_TPR_GAP(preds, Y, group).update(compute_TNR_GAP(preds, Y, group) )

def compute_AccDisp(preds, Y, group):
    return compute_ACC_GAP(preds, Y, group)

def compute_fairness(predss,Y,group):
    res = compute_TPR_GAP(preds, Y, group)
    res.update(compute_TNR_GAP(preds, Y, group))
    res.update(compute_ACC_GAP(preds,Y,group))
    return res


In [ ]:
epsilon_test = np.concatenate([np.linspace(0,0.5,30),[1]])

In [ ]:
results = []
for i in range(1):
    print('***** FOLD %d ******'%i)
    train  = pd.read_csv('split_data/bin_compas_train_%d.csv'%i)
    test = pd.read_csv('split_data/bin_compas_test_%d.csv'%i)

    for fairMet in [4]:
        for eps in epsilon_test:
            print('Epsilon: %f'%eps)
            
            start_time = time.time()
            c = CorelsClassifier(n_iter=1000000, c=0.001, fairness = fairMet, epsilon = eps, 
                                 maj_vect = train['race_True.1'].to_numpy())
            a = c.fit(train.drop('Y',axis=1).to_numpy(), train['Y'].to_numpy())
            end_time = time.time() - start_time
            
            preds = c.predict(test.drop('Y',axis=1).to_numpy()).astype(np.bool)
            y = test['Y'].to_numpy()
            group = test['race_True.1'].to_numpy()
            
            res = compute_fairness(preds, y, group)
            res['epsilon'] = eps
            res['fold'] = i
            res['fairnessCriteria'] = fairMet
            res['algo'] = 'fairCorels'
            res['train_time'] = end_time
            results.append(res)

In [ ]:
pd.DataFrame.from_records(results).to_csv('fairCorels_results.csv', index=False)